In [6]:
import os
import findspark
spark_home="C:\Users\minkyu\Downloads\spark-1.6.0-bin-hadoop2.6"
findspark.init(spark_home)

In [7]:
import pyspark
conf=pyspark.SparkConf()
conf=pyspark.SparkConf().setAppName("myAppName")
sc = pyspark.SparkContext(conf=conf)

In [13]:
from pyspark.sql import SQLContext
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer
from pyspark.sql import SQLContext
sqlCtx=SQLContext(sc)

df = sqlCtx.createDataFrame(
    [
        [1, u'서울시 승용차요일제의 시행효과분석 및 발전방안을 추진하기 위해서'],
        [1, u'승용차요일제 참여자들의 참여방식 및 문제점을 분석'],
        [1, u'개선방안의 마련 등을 위해 설문조사를 실시하고자 합니다.'],
        [1, u'바쁘시더라도 잠시 시간을 내시어 효율적이고 바람직한 정책이 수립되도록'],
        [1, u'진솔하게 작성해 주시면 감사 하겠습니다.'],
        [0, u'안녕하세요? 서울시입니다.'],
        [0, u'서울시는 노인 인구의 증가 추세에 따라 노인성질환 치료를 위한'],
        [0, u'전문적인 공공의료서비스 제공을 위해 양천구 신정동에'],
        [0, u'노인성질환 전문치료병원을 건립 중에 있습니다.'],
        [0, u'이에 서울시에서는 노인성질환 전문치료병원에 걸맞는 명칭을 제정하고자'],
        [0, u'명칭 선호도 조사를 실시하고 있습니다.'],
        [0, u'응답하신 내용은 통계분석 목적으로만 활용되오니'],
        [0, u'바쁘시더라도 설문에 적극 참여해주시면 대단히 감사하겠습니다. '],
    ],
    ['cls','sent']
)

In [14]:
df.printSchema()

root
 |-- cls: long (nullable = true)
 |-- sent: string (nullable = true)



In [15]:
tokenizer = Tokenizer(inputCol="sent", outputCol="words")

In [16]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")

In [18]:
stopwords=list()
_mystopwords=[u"위해서",u"합니다.",u"있습니다.", u"하겠습니다."]
for e in _mystopwords:
    stopwords.append(e)

In [20]:
for e in stop.getStopWords():
    print e,

위해서 합니다. 있습니다. 하겠습니다.


In [21]:
tokDf = tokenizer.transform(df)
stopDf=stop.transform(tokDf)
stopDf.show()

+---+--------------------+--------------------+--------------------+
|cls|                sent|               words|             nostops|
+---+--------------------+--------------------+--------------------+
|  1|서울시 승용차요일제의 시행효과분...|[서울시, 승용차요일제의, 시행...|[서울시, 승용차요일제의, 시행...|
|  1|승용차요일제 참여자들의 참여방식...|[승용차요일제, 참여자들의, 참...|[승용차요일제, 참여자들의, 참...|
|  1|개선방안의 마련 등을 위해 설문...|[개선방안의, 마련, 등을, 위...|[개선방안의, 마련, 등을, 위...|
|  1|바쁘시더라도 잠시 시간을 내시어...|[바쁘시더라도, 잠시, 시간을,...|[바쁘시더라도, 잠시, 시간을,...|
|  1|진솔하게 작성해 주시면 감사 하...|[진솔하게, 작성해, 주시면, ...|[진솔하게, 작성해, 주시면, 감사]|
|  0|      안녕하세요? 서울시입니다.|   [안녕하세요?, 서울시입니다.]|   [안녕하세요?, 서울시입니다.]|
|  0|서울시는 노인 인구의 증가 추세...|[서울시는, 노인, 인구의, 증...|[서울시는, 노인, 인구의, 증...|
|  0|전문적인 공공의료서비스 제공을 ...|[전문적인, 공공의료서비스, 제...|[전문적인, 공공의료서비스, 제...|
|  0|노인성질환 전문치료병원을 건립 ...|[노인성질환, 전문치료병원을, ...|[노인성질환, 전문치료병원을, ...|
|  0|이에 서울시에서는 노인성질환 전...|[이에, 서울시에서는, 노인성질...|[이에, 서울시에서는, 노인성질...|
|  0|명칭 선호도 조사를 실시하고 있...|[명칭, 선호도, 조사를, 실시...|[명칭, 선호도, 조사를, 실시하고]|
|  0|응답하신 내용은 통계분석 목적으...|[응답하신, 내

In [22]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="nostops", outputCol="cv", vocabSize=30,minDF=1.0)
cvModel = cv.fit(stopDf)
cvDf = cvModel.transform(stopDf)

cvDf.collect()
cvDf.select('words','nostops','cv').show()

+--------------------+--------------------+--------------------+
|               words|             nostops|                  cv|
+--------------------+--------------------+--------------------+
|[서울시, 승용차요일제의, 시행...|[서울시, 승용차요일제의, 시행...|(30,[3,20],[1.0,1...|
|[승용차요일제, 참여자들의, 참...|[승용차요일제, 참여자들의, 참...|(30,[3,12,17,18],...|
|[개선방안의, 마련, 등을, 위...|[개선방안의, 마련, 등을, 위...|(30,[1,4],[1.0,1.0])|
|[바쁘시더라도, 잠시, 시간을,...|[바쁘시더라도, 잠시, 시간을,...|(30,[2,7,8,15,24]...|
|[진솔하게, 작성해, 주시면, ...|[진솔하게, 작성해, 주시면, 감사]|     (30,[23],[1.0])|
|   [안녕하세요?, 서울시입니다.]|   [안녕하세요?, 서울시입니다.]|     (30,[16],[1.0])|
|[서울시는, 노인, 인구의, 증...|[서울시는, 노인, 인구의, 증...|(30,[0,9,19,25,26...|
|[전문적인, 공공의료서비스, 제...|[전문적인, 공공의료서비스, 제...|(30,[1,6,27],[1.0...|
|[노인성질환, 전문치료병원을, ...|[노인성질환, 전문치료병원을, ...|      (30,[0],[1.0])|
|[이에, 서울시에서는, 노인성질...|[이에, 서울시에서는, 노인성질...|(30,[0,5,22],[1.0...|
|[명칭, 선호도, 조사를, 실시...|[명칭, 선호도, 조사를, 실시하고]|     (30,[10],[1.0])|
|[응답하신, 내용은, 통계분석,...|[응답하신, 내용은, 통계분석,...|(30,[11,13,14],[1...|
|[바쁘시더라도, 설문에, 적극,...|[바쁘

In [23]:
cvDf.select('cv').take(13)

[Row(cv=SparseVector(30, {3: 1.0, 20: 1.0})),
 Row(cv=SparseVector(30, {3: 1.0, 12: 1.0, 17: 1.0, 18: 1.0})),
 Row(cv=SparseVector(30, {1: 1.0, 4: 1.0})),
 Row(cv=SparseVector(30, {2: 1.0, 7: 1.0, 8: 1.0, 15: 1.0, 24: 1.0})),
 Row(cv=SparseVector(30, {23: 1.0})),
 Row(cv=SparseVector(30, {16: 1.0})),
 Row(cv=SparseVector(30, {0: 1.0, 9: 1.0, 19: 1.0, 25: 1.0, 26: 1.0, 29: 1.0})),
 Row(cv=SparseVector(30, {1: 1.0, 6: 1.0, 27: 1.0})),
 Row(cv=SparseVector(30, {0: 1.0})),
 Row(cv=SparseVector(30, {0: 1.0, 5: 1.0, 22: 1.0})),
 Row(cv=SparseVector(30, {10: 1.0})),
 Row(cv=SparseVector(30, {11: 1.0, 13: 1.0, 14: 1.0})),
 Row(cv=SparseVector(30, {2: 1.0, 21: 1.0, 28: 1.0}))]